In [1]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
import time
import pandas as pd
import pickle
from multiprocessing import Process
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool, ECCConv
from spektral.transforms.normalize_adj import NormalizeAdj
from spektral.utils import reorder
from ipywidgets import Checkbox, Dropdown, Accordion, VBox
import sklearn.metrics as metrics
from sklearn.calibration import calibration_curve
from spektral.data import Dataset, Graph, DisjointLoader
from spektral.layers import CrystalConv, GlobalAvgPool
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

import copy
import logging
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import random
import sys
import tensorflow as tf
import requests
from os.path import isfile
import time

In [2]:
node_feat = pd.read_pickle('simple_nodes.pkl')
adj_mat = pd.read_pickle('adj_mat.pkl')
edge_feat = pd.read_pickle('edge_features.pkl')
y = pd.read_pickle('y.pkl')

In [3]:
edge_feat = adj_mat * edge_feat

In [4]:
y = y.reset_index()
y['uniqueplayId'] = y['uniqueplayId'].astype(int)
y = y.set_index(['uniqueplayId','frameId'])

In [5]:
node_feat = node_feat.reset_index()
node_feat['uniqueplayId'] = node_feat['uniqueplayId'].astype(int)
node_feat = node_feat.set_index(['uniqueplayId','frameId','nflId'])
node_feat.isna().sum()

new_x                0
new_y                0
Defense              0
o                    0
score_d              0
frames_after_snap    0
dtype: int64

In [6]:
def make_graph(index):
    ## PLAY_ID MUST BE A STRING
    
    play_id = index[0]
    frame_id = index[1]

    # Node features
    ## filter the node features matrix by given play id and frame id
    x_temp = node_feat.query(f'(uniqueplayId=={play_id})&(frameId=={frame_id})')
    x_temp = np.array(x_temp)
    #print(x_temp.shape)

    # Adjacency
    a_temp = adj_mat[(play_id, frame_id)]
    a_temp = sp.csr_matrix(a_temp)
    #print(a_temp.shape)
    
    # Edges
    ## get the correct edge matrix based on play id and frame id
    e_temp = edge_feat[(play_id, frame_id)]
    e_sp_mat = sp.find(e_temp)

    edge_indeces = np.array([e_sp_mat[0], e_sp_mat[1]]).T

    edge_vals = e_sp_mat[2]
    e_temp = reorder(edge_indeces, edge_features=edge_vals)[1].reshape(len(edge_vals), 1)
    #print(e_temp.shape)

    # Labels
    ## get the single label of coverage from y for that play id
    y_temp = y.query(f'(uniqueplayId=={play_id})&(frameId=={frame_id})').values[0]
    #print(y_temp.shape)

    return Graph(x=x_temp, a=a_temp, e=e_temp, y=y_temp)

In [7]:
make_graph(('202109090097', 6.0))

Graph(n_nodes=23, n_node_features=6, n_edge_features=1, n_labels=7)

In [10]:
%%time
################################################################################
# Load data
################################################################################

from concurrent.futures import ProcessPoolExecutor

class MyDataset(Dataset):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def read(self):
        
        all_graphs = []
        indeces = adj_mat.index

        with ProcessPoolExecutor(max_workers=40) as executor:
            for r in executor.map(make_graph, indeces):
                all_graphs.append(r)
            # We must return a list of Graph objects
        return all_graphs


data = MyDataset()

CPU times: user 2min 56s, sys: 49.2 s, total: 3min 46s
Wall time: 36min 32s


In [11]:
with open('complex_graphs.pkl', 'wb') as b:
    pickle.dump(data, b)

In [6]:
class MyDataset(Dataset):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def read(self):
        
        all_graphs = []
        indeces = edge_feat.index

        with ProcessPoolExecutor(max_workers=40) as executor:
            for r in executor.map(make_graph, indeces):
                all_graphs.append(r)
            # We must return a list of Graph objects
        return all_graphs
    
data = pd.read_pickle('complex_graphs.pkl')

In [7]:
test_data = data[150959:]
data = data[:150959]

In [8]:
# Train/valid/test split
idxs = np.random.permutation(len(data))
split_va = int(0.9 * len(data))
idx_tr, idx_va = np.split(idxs, [split_va])
data_tr = data[idx_tr]
data_va = data[idx_va]

In [9]:
learning_rate = 0.01  # Learning rate
epochs = 400  # Number of training epochs
es_patience = 25  # Patience for early stopping
batch_size = 32  # Batch size
layers = 3 # Number of CrystalConv layers
channels = 128 # Number of hidden nodes
n_out = 7

In [10]:
# Data loaders
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = DisjointLoader(data_va, batch_size=batch_size)
loader_te = DisjointLoader(test_data, batch_size=batch_size, shuffle = False)

In [11]:
class GNN(Model):
    '''
    Building the Graph Neural Network configuration with Model as the parent class 
    from spektral library.
    '''
    def __init__(self, n_layers):
        '''
        Constructor code for setting up the layers needed for training the model.
        '''
        super().__init__()
        self.conv1 = CrystalConv()
        self.convs = []
        for _ in range(1, n_layers):
            self.convs.append(
                CrystalConv()
            )
        self.pool = GlobalAvgPool()
        self.dense1 = Dense(channels, activation = tf.keras.layers.LeakyReLU(alpha = 0.1))
        self.dropout = Dropout(0.5)
        self.dense2 = Dense(channels, activation = tf.keras.layers.LeakyReLU(alpha = 0.1))
        self.dense3 = Dense(n_out, activation="softmax")

    def call(self, inputs):
        '''
        Build the neural network.
        '''
        x, a, e, i = inputs
        x = self.conv1([x, a, e])
        for conv in self.convs:
            x = conv([x, a, e])
        x = self.pool([x, i])
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        return self.dense3(x)
    
model = GNN(layers)
optimizer = Adam(learning_rate=learning_rate)
loss_fn = CategoricalCrossentropy()

In [12]:
def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        #print(target)
        #print(inputs)
        #print(pred)
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])

In [13]:
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc

In [20]:
%%time

epoch = step = 0
best_val_loss = np.inf
best_weights = None
patience = es_patience
results = []
for batch in loader_tr:
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))
    if step == loader_tr.steps_per_epoch:
        step = 0
        epoch += 1

        # Compute validation loss and accuracy
        val_loss, val_acc = evaluate(loader_va)
        print(
            "Ep. {} - Loss: {:.3f} - Acc: {:.3f} - Val loss: {:.3f} - Val acc: {:.3f}".format(
                epoch, *np.mean(results, 0), val_loss, val_acc
            )
        )

        # Check if loss improved for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = es_patience
            print("New best val_loss {:.3f}".format(val_loss))
            best_weights = model.get_weights()
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping (best val_loss: {})".format(best_val_loss))
                break
        results = []

/home/can2hr/.local/lib/python3.8/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
/home/can2hr/.local/lib/python3.8/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Ep. 1 - Loss: 1.873 - Acc: 0.318 - Val loss: 1.637 - Val acc: 0.342
New best val_loss 1.637
Ep. 2 - Loss: 1.648 - Acc: 0.341 - Val loss: 1.581 - Val acc: 0.364
New best val_loss 1.581
Ep. 3 - Loss: 1.625 - Acc: 0.351 - Val loss: 1.592 - Val acc: 0.368
Ep. 4 - Loss: 1.647 - Acc: 0.363 - Val loss: 1.866 - Val acc: 0.153
Ep. 5 - Loss: 1.787 - Acc: 0.321 - Val loss: 1.660 - Val acc: 0.328
Ep. 6 - Loss: 1.642 - Acc: 0.338 - Val loss: 1.533 - Val acc: 0.397
New best val_loss 1.533
Ep. 7 - Loss: 2.102 - Acc: 0.324 - Val loss: 1.701 - Val acc: 0.338
Ep. 8 - Loss: 1.639 - Acc: 0.358 - Val loss: 1.491 - Val acc: 0.419
New best val_loss 1.491
Ep. 9 - Loss: 1.489 - Acc: 0.401 - Val loss: 1.376 - Val acc: 0.422
New best val_loss 1.376
Ep. 10 - Loss: 1.422 - Acc: 0.425 - Val loss: 1.331 - Val acc: 0.483
New best val_loss 1.331
Ep. 11 - Loss: 1.410 - Acc: 0.434 - Val loss: 1.266 - Val acc: 0.491
New best val_loss 1.266
Ep. 12 - Loss: 1.401 - Acc: 0.447 - Val loss: 1.349 - Val acc: 0.476
Ep. 13 - Loss

In [21]:
model.set_weights(best_weights) 
model.save('saved_model/complex_best_model')

INFO:tensorflow:Assets written to: saved_model/complex_best_model/assets


INFO:tensorflow:Assets written to: saved_model/complex_best_model/assets


In [11]:
model = tf.keras.models.load_model('saved_model/complex_best_model', compile = False)

/home/can2hr/.local/lib/python3.8/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [23]:
test_loss, test_acc = evaluate(loader_te)
print("Done. Test loss: {:.4f}. Test acc: {:.2f}".format(test_loss, test_acc))

Done. Test loss: 1.1517. Test acc: 0.57


In [12]:
def evaluate_play(loader):
    true = []
    predict = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        true.append(target)
        predict.append(pred)
    return true, predict

In [13]:
true, predict = evaluate_play(loader_te)

In [14]:
predictions = pd.DataFrame()
for batch in predict:
    for i in batch:
        pred = pd.DataFrame(i).T
        predictions = pd.concat([predictions, pred])

In [27]:
predictions['predicted_coverage'] = predictions.idxmax(axis = 1)
coverage = pd.DataFrame(y[150959:].idxmax(axis=1)).rename(columns = {0:'coverage'})
coverage['predicted_coverage'] = predictions['predicted_coverage'].values
sum(coverage['predicted_coverage'] == coverage['coverage'])/len(coverage)

0.5677795442501324

In [28]:
coverage = coverage.reset_index()

In [29]:
counts = pd.DataFrame(coverage.groupby(['uniqueplayId','predicted_coverage']).size()).rename(columns = {0:'count'})

In [30]:
counts = counts.loc[counts.groupby(['uniqueplayId'])["count"].idxmax()]

In [31]:
counts = counts.reset_index()

In [32]:
results = pd.DataFrame(coverage.groupby('uniqueplayId').first()['coverage']).reset_index()
results = results.merge(counts, how = 'left')

In [33]:
sum(results['predicted_coverage'] == results['coverage'])/len(results)

0.6035211267605634

In [18]:
## Separate accuracy calculation method

In [38]:
coverage = pd.DataFrame(y[150959:].idxmax(axis=1)).rename(columns = {0:'coverage'})

In [39]:
predictions

,0,1,2,3,4,5,6,temp
0,0.002279,0.039442,0.491626,0.030557,0.099340,0.080536,0.256220,0
0,0.002851,0.041547,0.460751,0.037469,0.121662,0.068214,0.267506,1
0,0.003382,0.043076,0.425344,0.047913,0.152193,0.053128,0.274963,2
0,0.001650,0.032414,0.544076,0.016310,0.058394,0.119064,0.228092,3
0,0.002097,0.033154,0.518805,0.019712,0.074321,0.102649,0.249262,4
...,...,...,...,...,...,...,...,...
0,0.041752,0.066558,0.279795,0.000385,0.017510,0.545954,0.048046,37735
0,0.060969,0.114076,0.270709,0.001016,0.024884,0.472626,0.055720,37736
0,0.046937,0.061138,0.369192,0.001186,0.045648,0.386835,0.089064,37737
0,0.038622,0.048735,0.374062,0.000804,0.043594,0.406255,0.087929,37738


In [40]:
coverage['temp'] = np.arange(len(coverage))
predictions['temp'] = np.arange(len(predictions))

In [41]:
coverage = coverage.reset_index().merge(predictions, how = 'left').set_index(['uniqueplayId','frameId','coverage']).drop('temp', axis = 1)

In [42]:
probs = coverage.groupby('uniqueplayId').apply(lambda x: x.sum()/len(x))

In [43]:
probs = pd.DataFrame(probs.idxmax(axis = 1)).rename(columns = {0:'prediction'}).reset_index()
probs

,uniqueplayId,prediction
0,202110170483,3
1,202110170673,3
2,202110170762,1
3,202110170856,3
4,202110170955,4
...,...,...
1415,20211025003684,2
1416,20211025003735,3
1417,20211025003904,4
1418,20211025003926,2


In [44]:
plays = pd.DataFrame(y[150959:].idxmax(axis=1)).rename(columns = {0:'coverage'})

In [45]:
results = pd.DataFrame(plays.groupby('uniqueplayId').first()).reset_index()
results = results.merge(probs, how = 'left')

In [46]:
results

,uniqueplayId,coverage,prediction
0,202110170483,3,3
1,202110170673,3,3
2,202110170762,1,1
3,202110170856,3,3
4,202110170955,6,4
...,...,...,...
1415,20211025003684,1,2
1416,20211025003735,3,3
1417,20211025003904,4,4
1418,20211025003926,2,2


In [47]:
results['values'] = str('coverage') + str('prediction')

In [48]:
results

,uniqueplayId,coverage,prediction,values
0,202110170483,3,3,coverageprediction
1,202110170673,3,3,coverageprediction
2,202110170762,1,1,coverageprediction
3,202110170856,3,3,coverageprediction
4,202110170955,6,4,coverageprediction
...,...,...,...,...
1415,20211025003684,1,2,coverageprediction
1416,20211025003735,3,3,coverageprediction
1417,20211025003904,4,4,coverageprediction
1418,20211025003926,2,2,coverageprediction


In [62]:
sum(results['prediction'] == results['coverage'])/len(results)

0.6330985915492958

In [15]:
## Checking performance just looking at the first frame

In [22]:
coverage = pd.DataFrame(y[150959:].idxmax(axis=1)).rename(columns = {0:'coverage'})

In [23]:
coverage['temp'] = np.arange(len(coverage))
predictions['temp'] = np.arange(len(predictions))

In [24]:
coverage = coverage.reset_index().merge(predictions, how = 'left').set_index(['uniqueplayId','frameId']).drop('temp', axis = 1)

In [36]:
bayes = pd.DataFrame(coverage.groupby('uniqueplayId').first().reset_index().set_index(['uniqueplayId','coverage']).idxmax(axis=1)).rename(columns = {0:'prediction'}).reset_index()

In [37]:
sum(bayes['prediction'] == bayes['coverage'])/len(bayes)

0.46056338028169014